### Make plots and statistics for selected SNPs
Upload tables done in the previous step

In [1]:
setwd('/frazer01/home/paola/Family1070/private_output/Validation')
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(gplots))
suppressPackageStartupMessages(library(vioplot))
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(beeswarm))
suppressPackageStartupMessages(library(limma))

In [2]:
gene_info<-read.table("/publicdata/gencode_v19_20151104/gene_info.tsv", header=T, sep="\t")

In [3]:
rna_cm  = read.csv   ("data_tables/Production_cm_samples_wgsid.csv", row.names=1)
gt      = read.csv   ("data_tables/Cardips_NKX25_ase_genotypes_coded.csv", check.names=F, row.names=1)

In [4]:
gt<-t(gt)
colnames(gt)<-gt["ID",]
gt<-gt[rownames(gt) %in% unique(as.character(rna_cm$wgs_id)),]
gt_cm<-merge(rna_cm, gt, by.x="wgs_id", by.y="row.names")

In [15]:
#### This is the one normalized expression calclucalte using vst and correction for cTNT
rna_exp = read.table ("data_tables/Production_cms_vst_counts_corrected_ctnt.txt", header=T, check.names=F, row.names=1)
rna_exp<-subset(rna_exp, select=as.character(gt_cm$assay_uuid))

In [33]:
#### New DATA from MATTEO - normalized for peer factors # the colnames is wgs_id
rna_exp = read.table ("matteo_tables_eqtl/peer_residuals.txt", header=T, check.names=F, row.names=1)
meta_matteo = read.table("matteo_tables_eqtl/metadata.txt", header=T, sep="\t")
rna_exp = subset(rna_exp, select = as.character(meta_matteo$assay_uuid))
gt_cm = subset(gt_cm,assay_uuid %in% colnames(rna_exp) )
rna_exp = subset(rna_exp, select = as.character(gt_cm$assay_uuid))
dim(rna_exp)

[1] 16177   164

In [5]:
#### Newer DATA from MATTEO - normalized for peer factors # the colnames is wgs_id
rna_exp = read.table ("matteo_tables_eqtl/tpm_norm.txt", header=T, check.names=F, row.names=1)
meta_matteo = read.table("matteo_tables_eqtl/metadata.txt", header=T, sep="\t")
rna_exp = subset(rna_exp, select = as.character(meta_matteo$assay_uuid))
gt_cm = subset(gt_cm,assay_uuid %in% colnames(rna_exp) )
rna_exp = subset(rna_exp, select = as.character(gt_cm$assay_uuid))
dim(rna_exp)

[1] 16177   164

In [6]:
#### Only one CM per person
gt_cm_uni=gt_cm[order(gt_cm$cTNT_TPM, decreasing=T),]
gt_cm_uni=gt_cm_uni[!duplicated(gt_cm_uni$subject_uuid),]
dim(gt_cm_uni)

[1]  128 1977

In [7]:
plot_eQTLs= function(exp_mat, gt_mat, genes, type){
    
     
    for( i in 1:nrow(genes)){

    snp <-as.character(genes$snp)[i]
    gene_exp <- exp_mat[as.character(genes$gene_id[i]),]
    genotype<-as.numeric(as.character(gt_mat[,snp]))
    
   
    dt<-data.frame(t(gene_exp), genotype)   # maybe scale---does not cahnage the pval
    colnames(dt)<-c('gene_exp', 'genotype')
   
        
   l<-lm(gene_exp~genotype, dt)                         
    
    if (type=="boxplot"){
    boxplot(gene_exp~genotype, dt, ylab=paste(genes$gene_name[i], "expression"), xlab=paste(snp, "genotype"),
           main=(paste("p-val=", signif(summary(l)$coefficients[2,4]))), las=1)
    #stripchart(gene_exp~genotype, dt, method="jitter", add=T, vertical=T, pch=1, col="steelblue")
        beeswarm(gene_exp~genotype, dt, log = F, pch = 21, col = "steelblue",corral="gutter", add = TRUE, cex=0.8)
    mtext(paste("n=", aggregate(gene_exp~genotype, dt, length)[,2]) , side = 3, outer = FALSE, at = 1:length(unique(dt$genotype)),
           adj = NA, padj = NA, cex = 0.8, col = NA)
    }
        
        if (type=="beeswarm"){
            
          beeswarm(gene_exp~genotype, dt, 
          log = F, pch = 21, col = "steelblue",corral="random",
          ylab=paste(genes$gene_name[i], "expression"), xlab=paste(snp, "genotype"),
          main=(paste("p-val=", signif(summary(l)$coefficients[2,4]))), las=1)
          bxplot(gene_exp~genotype, dt, add = TRUE)
          mtext(paste("n=", aggregate(gene_exp~genotype, dt, length)[,2]) , side = 1, outer = FALSE, at = 1:length(unique(dt$genotype)),
          adj = NA, padj = NA, cex = 0.8, col = NA)
          mtext(genes$gene_id[i] , side = 3, outer = FALSE, at = length(unique(dt$genotype))-1,
           adj = NA, padj = NA, cex = 0.8, col = NA)
  
        }

}
    }


In [8]:
### Input table for the function
snps<-c( 'rs6801957','rs7612445','rs7612445','rs7612445','rs7612445','rs7612445','rs7612445',
        'rs4890490','rs6606689','rs6606689','rs6606689','rs7132327','rs7132327','rs3807989','rs3807989',
        'rs7986508', 'rs35176054', 'rs590041','rs590041','rs590041',
       "rs4657167", "rs8044595","rs8044595","rs8044595" , "rs8044595", "rs10841486",
        "rs6569252", "rs7850041", "rs176107", "rs3943037", "rs56248930",
      'rs13006682', 'rs2267469')
gene_name<-c("SCN5A", 'GNB4', 'MFN1', 'NDUFB5', 'PIK3CA', 'ZNF639','RP11-145M9.4', 
             'SETBP1','PPTC7',"ARPC3", "MYL2",'TBX3','TBX5','CAV1',"CAV2",
              'LRCH1', 'SH3PXD2A','SSBP3', 'MRPL37', 'MROH7',
              "NOS1AP", "MYH11", "NOMO3","ABCC1","ABCC6", "PDE3A", "GJA1" , "PPAPDC3", "MEF2C", "SHROOM3", "SHROOM3",
            'FHL2', 'PACSIN2')
test_genes=data.frame(snps, gene_name)
test_genes = merge(test_genes, gene_info, by="gene_name")

In [25]:
### Input table for the function
# snps<-c(  'rs35176054')
# gene_name<-c(  "SH3PXD2A")
# test_genes=data.frame(snps, gene_name)
# test_genes = merge(test_genes, gene_info, by="gene_name")

In [9]:
rna_exp_uni<-subset(rna_exp, select=as.character(gt_cm_uni$assay_uuid))
pdf("Association_unique_subjects_CMs_genes_new_peers_andcovariates.pdf")
par(mfrow=c(3,3))
plot_eQTLs(rna_exp_uni, gt_cm_uni, test_genes, type="boxplot")
dev.off()

png 
  2